In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from sklearn.model_selection import GridSearchCV, train_test_split, cross_val_score, ShuffleSplit, KFold
from sklearn import metrics
import keras
from keras.layers import Dense, Conv1D, GlobalAveragePooling1D, GlobalMaxPooling1D
from keras.layers.recurrent import LSTM
from keras.layers.wrappers import TimeDistributed, Bidirectional
from keras.models import Sequential
from keras.preprocessing.sequence import pad_sequences
import word2vec

/Users/prihodad/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
TIMESTEPS = 128
BATCH_SIZE = 16
PFAM2VEC_DIMENSIONS = 100

In [3]:
labels = pd.read_csv("../data/training/positive/mibig_bgcs_all.classes.csv").set_index('contig_id')
labels = labels[~labels['?'].astype(np.bool)]
del labels['?']
del labels['Nucleoside']
NUM_LABELS = labels.shape[1]
labels.sample(5)

,Alkaloid,NRP,Other,Polyketide,RiPP,Saccharide,Terpene
contig_id,,,,,,,
BGC0001319.1,0,0,0,0,0,0,1
BGC0000201.1,0,0,0,1,0,1,0
BGC0001243.8,0,0,0,1,0,0,0
BGC0001192.1,0,1,0,0,0,0,0
BGC0001219.1,0,0,0,1,0,0,0


In [5]:
pfam2vec_bin = word2vec.load('../data/features/pfam2vec-experiments/pfam2vec_top.bin', kind='bin')
pfam2vec = pd.DataFrame(pfam2vec_bin.vectors, index=pfam2vec_bin.vocab)
pfam2vec.head(2)

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
</s>,0.143333,0.158255,-0.137158,-0.117384,0.048936,0.108183,0.033691,0.007568,-0.129039,0.079442,...,-0.054848,-0.075464,-0.104101,0.100795,0.056553,0.015217,-0.078551,0.054569,0.109554,0.006934
PF00005,0.045443,-0.035938,0.100785,0.040616,-0.164825,0.073102,0.058317,-0.108120,-0.069214,0.148570,...,-0.182994,0.114296,-0.118334,0.086235,0.099875,-0.009867,-0.118115,0.002825,0.037026,-0.107635


In [6]:
domains = pd.read_csv('../data/training/positive/mibig_bgcs_all.csv')

In [7]:
contig_ids = domains['contig_id'].unique()
contig_ids = list(np.intersect1d(contig_ids, labels.index))
len(contig_ids)

1535

In [8]:
def make_batch_size_divisible(vectors):
    missing_to_divisible = BATCH_SIZE - (len(vectors) % BATCH_SIZE)
    shape = list(vectors.shape)
    shape[0] = missing_to_divisible
    return np.concatenate([vectors, np.zeros(shape=shape)])


In [41]:
y_samples = labels.reindex(contig_ids).values
y_samples = pd.DataFrame(make_batch_size_divisible(y_samples), columns=labels.columns)
print(y_samples.shape)
y_samples.head()

(1536, 7)


,Alkaloid,NRP,Other,Polyketide,RiPP,Saccharide,Terpene
0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2,0.0,0.0,0.0,1.0,0.0,0.0,0.0
3,0.0,0.0,0.0,1.0,0.0,0.0,0.0
4,0.0,0.0,0.0,1.0,0.0,0.0,0.0


In [22]:
domain_vectors = pfam2vec.reindex(domains['pfam_id'])
domain_vectors['contig_id'] = domains['contig_id'].values
sample_vectors = domain_vectors.groupby('contig_id')
X_samples = [sample_vectors.get_group(contig_id).drop('contig_id', axis=1).dropna() for contig_id in contig_ids]
X_samples = pad_sequences(X_samples, maxlen=TIMESTEPS, dtype=np.float, padding='post', truncating='post')
X_samples = make_batch_size_divisible(X_samples)
#X_samples = X_samples.reshape(-1, BATCH_SIZE, TIMESTEPS, PFAM2VEC_DIMENSIONS)
print(X_samples.shape)
print('First sample shape:', X_samples[0].shape)
X_samples[0]

(1536, 128, 100)
First sample shape: (128, 100)


array([[ 0.03427664, -0.0197117 ,  0.10328925, ...,  0.08086994,
         0.02974272, -0.13709141],
       [ 0.06393322, -0.0231906 ,  0.09249122, ...,  0.06740983,
        -0.00232806, -0.09247406],
       [ 0.09635212, -0.0401225 ,  0.0989542 , ...,  0.1043229 ,
         0.00819657, -0.17837451],
       ...,
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ]])

In [28]:
major_bgc_ids = pd.Series([contig_id.split('.')[0] for contig_id in contig_ids])
major_to_minor_ids = pd.Series(range(0, len(contig_ids)), index=major_bgc_ids)

In [29]:
unique_major_ids = major_bgc_ids.unique()
len(unique_major_ids)

1388

In [30]:
def create_lstm(stacked_sizes=[], batched=True):
    model = Sequential()
    args = {}
    if batched:
        args['batch_input_shape'] = (BATCH_SIZE, TIMESTEPS, 100)
    else:
        args['input_shape'] = (TIMESTEPS, 100)
    model.add(Bidirectional(
        layer=LSTM(
            units=16,
            return_sequences=False,
            dropout=0.2,
            recurrent_dropout=0.2,
            stateful=False
        ),
        **args
    ))
    for size in stacked_sizes:
        model.add(Bidirectional(layer=LSTM(units=size, return_sequences=False, stateful=False)))
    model.add(Dense(NUM_LABELS, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer="adam", metrics=[])
    return model

In [31]:
def create_conv1D(stacked_sizes=[], batched=True):
    model = Sequential()
    args = {}
    if batched:
        args['batch_input_shape'] = (BATCH_SIZE, TIMESTEPS, 100)
    else:
        args['input_shape'] = (TIMESTEPS, 100)
    model.add(Conv1D(128, 3, activation='relu', **args))
    model.add(Conv1D(64, 3, activation='relu'))
    model.add(GlobalMaxPooling1D())
    model.add(Dense(7, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer="adam", metrics=[])
    return model

In [44]:
def evaluate(create_model, epochs=200):
    splitter = KFold(n_splits=5, shuffle=True, random_state=0)

    scores = []
    accuracies = []
    for id_train_idx, id_test_idx in splitter.split(unique_major_ids):
        train_major_ids, test_major_ids = unique_major_ids[id_train_idx], unique_major_ids[id_test_idx]
        train_minor_ids, test_minor_ids = major_to_minor_ids.loc[train_major_ids], major_to_minor_ids.loc[test_major_ids]
        X_train, X_test = X_samples[train_minor_ids], X_samples[test_minor_ids]
        y_train, y_test = y_samples.loc[train_minor_ids].values, y_samples.loc[test_minor_ids].values
        X_train = make_batch_size_divisible(X_train)
        y_train = make_batch_size_divisible(y_train)
        print('Train:', len(X_train), 'Test:', len(X_test))
        train_model = create_model()
        model = create_model(batched=False)

        # fitting the model
        train_model.fit(X_train, y_train, epochs=epochs, verbose=2, batch_size=BATCH_SIZE)

        trained_weights = train_model.get_weights()
        model.set_weights(trained_weights)

        # predict the response
        pred = (model.predict(X_test) > 0.5).astype(np.int)
        aucs = metrics.roc_auc_score(y_test, pred, average=None)
        precisions = metrics.precision_score(y_test, pred, average=None)
        recalls = metrics.recall_score(y_test, pred, average=None)
        accuracy = metrics.accuracy_score(y_test, pred)
        accuracies.append(accuracy)
        scores += [{'Precision': p, 'Recall': r, 'AUC': a, 'Class': c, 'Samples': int(sum(y_samples[c]))} for p,r,a,c in zip(precisions, recalls, aucs, y_samples.columns)]

    scores = pd.DataFrame(scores).groupby('Class').mean().sort_values('Samples', ascending=False)
    print('Accuracy (exact match)', np.mean(accuracies))
    print(scores.mean())
    return scores

In [46]:
scores = evaluate(create_lstm, epochs=100)

Train: 1248 Test: 297
Epoch 1/100
 - 8s - loss: 2.0393
Epoch 2/100
 - 6s - loss: 1.7485
Epoch 3/100
 - 7s - loss: 1.6552
Epoch 4/100
 - 7s - loss: 1.5953
Epoch 5/100
 - 6s - loss: 1.5476
Epoch 6/100
 - 6s - loss: 1.4972
Epoch 7/100
 - 6s - loss: 1.4782
Epoch 8/100
 - 7s - loss: 1.4184
Epoch 9/100
 - 6s - loss: 1.3934
Epoch 10/100
 - 6s - loss: 1.3606
Epoch 11/100
 - 6s - loss: 1.3323
Epoch 12/100
 - 6s - loss: 1.3277
Epoch 13/100
 - 6s - loss: 1.3178
Epoch 14/100
 - 7s - loss: 1.2612
Epoch 15/100
 - 7s - loss: 1.2595
Epoch 16/100
 - 6s - loss: 1.2529
Epoch 17/100
 - 6s - loss: 1.2286
Epoch 18/100
 - 6s - loss: 1.2004
Epoch 19/100
 - 7s - loss: 1.1804
Epoch 20/100
 - 6s - loss: 1.1772
Epoch 21/100
 - 7s - loss: 1.2020
Epoch 22/100
 - 6s - loss: 1.1589
Epoch 23/100
 - 6s - loss: 1.1557
Epoch 24/100
 - 7s - loss: 1.1207
Epoch 25/100
 - 6s - loss: 1.1265
Epoch 26/100
 - 6s - loss: 1.1203
Epoch 27/100
 - 6s - loss: 1.1042
Epoch 28/100
 - 6s - loss: 1.0961
Epoch 29/100
 - 6s - loss: 1.0842
E

/Users/prihodad/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


Train: 1232 Test: 314
Epoch 1/100
 - 9s - loss: 2.0647
Epoch 2/100
 - 6s - loss: 1.7842
Epoch 3/100
 - 7s - loss: 1.5898
Epoch 4/100
 - 6s - loss: 1.4729
Epoch 5/100
 - 6s - loss: 1.4148
Epoch 6/100
 - 6s - loss: 1.3619
Epoch 7/100
 - 6s - loss: 1.3289
Epoch 8/100
 - 6s - loss: 1.3296
Epoch 9/100
 - 6s - loss: 1.2731
Epoch 10/100
 - 6s - loss: 1.2867
Epoch 11/100
 - 6s - loss: 1.2495
Epoch 12/100
 - 6s - loss: 1.2148
Epoch 13/100
 - 6s - loss: 1.2041
Epoch 14/100
 - 6s - loss: 1.1797
Epoch 15/100
 - 6s - loss: 1.2085
Epoch 16/100
 - 6s - loss: 1.1730
Epoch 17/100
 - 6s - loss: 1.1588
Epoch 18/100
 - 6s - loss: 1.1559
Epoch 19/100
 - 6s - loss: 1.1306
Epoch 20/100
 - 6s - loss: 1.1458
Epoch 21/100
 - 6s - loss: 1.1189
Epoch 22/100
 - 6s - loss: 1.1271
Epoch 23/100
 - 6s - loss: 1.1261
Epoch 24/100
 - 6s - loss: 1.1035
Epoch 25/100
 - 6s - loss: 1.0930
Epoch 26/100
 - 6s - loss: 1.0545
Epoch 27/100
 - 6s - loss: 1.0832
Epoch 28/100
 - 6s - loss: 1.0567
Epoch 29/100
 - 6s - loss: 1.0449
E

Epoch 41/100
 - 6s - loss: 0.9984
Epoch 42/100
 - 6s - loss: 0.9752
Epoch 43/100
 - 6s - loss: 0.9707
Epoch 44/100
 - 6s - loss: 0.9761
Epoch 45/100
 - 6s - loss: 0.9577
Epoch 46/100
 - 6s - loss: 0.9543
Epoch 47/100
 - 6s - loss: 0.9586
Epoch 48/100
 - 7s - loss: 0.9322
Epoch 49/100
 - 6s - loss: 0.9301
Epoch 50/100
 - 6s - loss: 0.9192
Epoch 51/100
 - 6s - loss: 0.9305
Epoch 52/100
 - 7s - loss: 0.8990
Epoch 53/100
 - 6s - loss: 0.9151
Epoch 54/100
 - 6s - loss: 0.9163
Epoch 55/100
 - 7s - loss: 0.9108
Epoch 56/100
 - 8s - loss: 0.9004
Epoch 57/100
 - 8s - loss: 0.9004
Epoch 58/100
 - 7s - loss: 0.8791
Epoch 59/100
 - 6s - loss: 0.8863
Epoch 60/100
 - 7s - loss: 0.8939
Epoch 61/100
 - 7s - loss: 0.8866
Epoch 62/100
 - 7s - loss: 0.8528
Epoch 63/100
 - 6s - loss: 0.8788
Epoch 64/100
 - 6s - loss: 0.8825
Epoch 65/100
 - 7s - loss: 0.8756
Epoch 66/100
 - 6s - loss: 0.8658
Epoch 67/100
 - 9s - loss: 0.8586
Epoch 68/100
 - 6s - loss: 0.8488
Epoch 69/100
 - 7s - loss: 0.8546
Epoch 70/100
 

In [47]:
scores

,AUC,Precision,Recall,Samples
Class,,,,
Polyketide,0.845845,0.905939,0.750234,652
NRP,0.791222,0.888066,0.612506,443
RiPP,0.916691,0.925897,0.843729,200
Saccharide,0.827766,0.877320,0.668039,180
Other,0.658116,0.757199,0.331750,174
Terpene,0.734303,0.853167,0.474937,120
Alkaloid,0.539710,0.360000,0.082051,39


In [48]:
scores.mean()

AUC            0.759093
Precision      0.795370
Recall         0.537606
Samples      258.285714
dtype: float64